In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using PyPlot
using Plots
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface;

In [ ]:
function CR3BPdynamics(rv) #Three body dynamics in Sun-Earth System

    μ = 9.537e-4
    
    r₁³= ((rv[1] + μ)^2.0     + rv[2]^2.0 + rv[3]^2.0)^1.5; # distance to m1, LARGER MASS
    r₂³= ((rv[1] - 1 + μ)^2.0 + rv[2]^2.0 + rv[3]^2.0)^1.5; # distance to m2, smaller mass
    # r₁³= ((x + μ)^2     + y^2 + z^2)^1.5; # distance to m1, LARGER MASS
    # r₂³= ((x - 1 + μ)^2 + y^2 + z^2)^1.5; # distance to m2, smaller mass

#     rvdot = zeros(6)
    rvdot = zeros(eltype(rv),6)
    rvdot[1:3] = [rv[4];rv[5];rv[6]]
    rvdot[4] = -((1.0 - μ)*(rv[1] + μ)/r₁³) - (μ*(rv[1] - 1.0 + μ)/r₂³) + 2.0*rv[5] + rv[1];
    rvdot[5] = -((1.0 - μ)*rv[2]      /r₁³) - (μ*rv[2]          /r₂³) - 2.0*rv[4] + rv[2];
    rvdot[6] = -((1.0 - μ)*rv[3]      /r₁³) - (μ*rv[3]          /r₂³);
    return rvdot
    # return [rv[4];rv[5];rv[6];-((1.0 - μ)*(rv[1] + μ)/r₁³) - (μ*(rv[1] - 1.0 + μ)/r₂³) + 2.0*rv[5] + rv[1];-((1.0 - μ)*rv[2]      /r₁³) - (μ*rv[2]          /r₂³) - 2.0*rv[4] + rv[2];-((1.0 - μ)*rv[3]      /r₁³) - (μ*rv[3]          /r₂³)]
end

In [ ]:
# function CR3BPdynamics(rv) #Three body dynamics in Sun-Earth System

#     μ = 9.537e-4
    
#     r₁³= ((rv[1] + μ)^2.0     + rv[2]^2.0 + rv[3]^2.0)^1.5; # distance to m1, LARGER MASS
#     r₂³= ((rv[1] - 1 + μ)^2.0 + rv[2]^2.0 + rv[3]^2.0)^1.5; # distance to m2, smaller mass
#     # r₁³= ((x + μ)^2     + y^2 + z^2)^1.5; # distance to m1, LARGER MASS
#     # r₂³= ((x - 1 + μ)^2 + y^2 + z^2)^1.5; # distance to m2, smaller mass

# #     rvdot = zeros(6)
#     rvdot = zeros(eltype(rv),6)
#     rvdot[1:3] = [rv[4];rv[5];rv[6]]
#     rvdot[4] = -((1.0 - μ)*(rv[1] + μ)/r₁³) - (μ*(rv[1] - 1.0 + μ)/r₂³) + 2.0*rv[5] + rv[1];
#     rvdot[5] = -((1.0 - μ)*rv[2]      /r₁³) - (μ*rv[2]          /r₂³) - 2.0*rv[4] + rv[2];
#     rvdot[6] = -((1.0 - μ)*rv[3]      /r₁³) - (μ*rv[3]          /r₂³);
#     # return rvdot
#     return [rv[4];rv[5];rv[6];-((1.0 - μ)*(rv[1] + μ)/r₁³) - (μ*(rv[1] - 1.0 + μ)/r₂³) + 2.0*rv[5] + rv[1];-((1.0 - μ)*rv[2]      /r₁³) - (μ*rv[2]          /r₂³) - 2.0*rv[4] + rv[2];-((1.0 - μ)*rv[3]      /r₁³) - (μ*rv[3]          /r₂³)]
# end
# function dircol_dynamics(x1,x2,h)
#     #Hermite-Simpson integration with first-order hold on u
#     f1 = CR3BPdynamics(x1) #Timestep k
#     f2 = CR3BPdynamics(x2) #Timestep k+1
#     xm = 0.5*(x1 + x2) + (h/8.0)*(f1 - f2) #
#     ẋm = (-3.0/(2.0*h))*(x1 - x2) - 0.25*(f1 + f2)
#     fm = CR3BPdynamics(xm)
#     return fm - ẋm
# end
# let
#     rv0 = randn(6)
#     xd = CR3BPdynamics(rv0)
#     # # first jacobian 
#     dynamics_jacobian = ForwardDiff.jacobian(CR3BPdynamics,rv0)
#     rv1 = randn(6)
#     h = 0.4
#     # # forward diff working on all inputs of the dircoldynamics function
#     A1 = ForwardDiff.jacobian(_x->dircol_dynamics(_x,rv1,h),rv0)
#     A2 = ForwardDiff.jacobian(_x->dircol_dynamics(rv0,_x,h),rv1)
#     A3 = ForwardDiff.derivative(_h->dircol_dynamics(rv0,rv1,_h),h)
# end

In [ ]:
h = 3.5 #10 Hz
Nx = 6     # number of state
Nu = 1     # number of controls
Tfinal = 180 # final time-50 knot points or something similar (Get Tfinal from approximation) 
Nt = Int(floor(Tfinal/h))+1   # number of time steps-101
thist = Array(range(0,h*(Nt-1), step=h));#0:1,step:0.1
n_nlp = (Nx+Nu)*Nt # number of decision variables-707
m_nlp = Nx*(Nt+1) #612
u_idx = [(Nx+Nu)*(i-1) .+ (Nx+1:Nx+Nu) for i = 1:Nt-1];
x_idx = [(Nx+Nu)*(i-1) .+ (1:Nx) for i = 1:Nt];
Uref = h*ones(Nt)

In [ ]:
#Non linear Approximation to third order expansion
function Non_lin_exp(tau)

    k = 3.2292680962;
    c2 = 4.0610735668;
    c3 = 3.0200105081;
    c4 = 3.0305378797;

    lam = sqrt((c2 + sqrt(9*c2^2 - 8*c2))/2);
    d1 = ((3*lam^2)/k)*(k*(6*lam^2 - 1) - 2*lam); 
    d2 = ((8*lam^2)/k)*(k*(11*lam^2 - 1) - 2*lam);
    a21 = 3*c3*(k^2 - 2)/(4*(1 + 2*c2)); 
    a22 = 3*c3/(4*(1 + 2*c2));
    a23 = -((3*c3*lam)/(4*k*d1))*(3*(k^3)*lam - 6*k*(k-lam) + 4); 
    a24 = -((3*c3*lam)/(4*k*d1))*(3*k*lam + 2);
    b21 = -3*c3*lam*(3*k*lam - 4)/(2*d1);
    b22 = -3*c3*lam/d1;
    a31 = (-9*lam/(4*d2))*(4*c3*(k*a23 - b21) + k*c4*(4 + k^2))  + ((9*lam^2 + 1 - c2)/(2*d2))*(3*c3*(2*a23 - k*b21) + c4*(2 + 3*k^2));
    d21 = -c3/(2*lam^2);
    d31 = (3/(64*lam^2))*(4*c3*a24 + c4);
    d32 = (3/(64*lam^2))*(4*c3*(a23 - d21) + c4*(4 + k^2));
    a32 = (-9*lam/(4*d2))*(4*c3*(3*k*a24 - b22) + k*c4) - 3*((9*lam^2 + 1 - c2)/(2*d2))*(c3*(k*b22 - d21 - 2*a24) - c4);
    b31 = (3/(8*d2))*8*lam*(3*c3*(k*b21 - 2*a23) - c4*(2 + 3*k^2)) + (3/(8*d2))*((9*lam^2 + 1 + 2*c2)*(4*c3*(k*a23 - b21)) + k*c4*(4 + k^2));
    b32 = ((9*lam)/d2)*(c3*(k*b22 + d21 + -2*a24) - c4)  + 3*((9*lam^2 + 1 + 2*c2)/(8*d2))*(4*c3*(k*a24 - b22) + k*c4);

    xpoint = 0.932385;
    ypoint = 0;
    zpoint = 0;

    phi = 0;
    w_p = 2.086453455;
    # tau = range(0,Tfinal,length = Nt)
    # tau_1 = w_p.*tau .+ phi;

    # tau = range(0,Tfinal,length = Nt)
    tau_1 = w_p*tau + phi;

    scaling_factor = 1.495978714e8;
    Ax = 206000/scaling_factor;
    Ay = 665000/scaling_factor;
    Az = 110000/scaling_factor;
    m=1 
    dm = 2 - m;
    x1 =    -Ax*cos(tau_1) + (a23*Ax^2 - a24*Az^2)*cos(2*tau_1) + (a31*Ax^3 - a32*Ax*Az^2)*cos(3*tau_1) + a21*Ax^2 + a22*Az^2 + xpoint;
    y1 =     Ay*sin(tau_1) + (b21*Ax^2 - b22*Az^2)*sin(2*tau_1) + (b31*Ax^3 - b32*Ax*Az^2)*sin(3*tau_1) + ypoint;
    z1 =  dm*Az*cos(tau_1) + dm*d21*Ax*Az*cos(2*tau_1 - 3) + dm*(d32*Az*Ax^2 - d31*Az^3)*cos(3*tau_1) + zpoint;
    # x1 =    -Ax.*cos.(tau_1) .+ (a23*Ax^2 - a24*Az^2).*cos.(2 .* tau_1) .+ (a31*Ax^3 - a32*Ax*Az^2).*cos.(3 .* tau_1) .+ a21*Ax^2 .+ a22*Az^2 .+ xpoint;
    # y1 =     Ay.*sin.(tau_1) .+ (b21*Ax^2 - b22*Az^2).*sin.(2 .* tau_1) .+ (b31*Ax^3 - b32*Ax*Az^2).*sin.(3 .* tau_1) .+ ypoint;
    # z1 =  dm*Az.*cos.(tau_1) .+ dm*d21*Ax*Az.*cos.(2 .* tau_1 .- 3) .+ dm*(d32*Az*Ax^2 - d31*Az^3).*cos.(3 .* tau_1) .+ zpoint;
    return [x1, y1, z1]
end 


In [ ]:
#Calc of Reference Trajectory 
tau = range(0,Tfinal,length = Nt);
const Xref_ =[zeros(Nx) for i=1:Nt];
for k = 1:Nt
    Pos = Non_lin_exp(tau[k]);
    Xref_[k][1] = Pos[1];
    Xref_[k][2] = Pos[2];
    Xref_[k][3] = Pos[3];
    Vels = ForwardDiff.derivative(Non_lin_exp,tau[k]);
    Xref_[k][4] = Vels[1];
    Xref_[k][5] = Vels[2];
    Xref_[k][6] = Vels[3];
end
const Xref_ = Xref_

const Uref = Uref
# @show ForwardDiff.jacobian(CR3BPdynamics, Xref_[5])
#Initial dt is the same as the Reference Traj 

In [ ]:
function dircol_dynamics(x1,x2,h)
    #Hermite-Simpson integration with first-order hold on u
    f1 = CR3BPdynamics(x1) #Timestep k
    f2 = CR3BPdynamics(x2) #Timestep k+1
    xm = 0.5.*(x1 + x2) .+ (h/8.0).*(f1 - f2) #
    ẋm = (-3.0/(2.0*h)).*(x1 - x2) .- 0.25.*(f1 + f2)
    fm = CR3BPdynamics(xm)
    return fm - ẋm
end

In [ ]:
# Cost weights
Q = Diagonal([1.0*ones(3); 1.0*ones(3)]);
R = 0.1;

In [ ]:
function stage_cost(x,u,Xref,Uref)
    return 0.5*((x-Xref)'*Q*(x-Xref)) + 0.5*(u-Uref)'*R*(u-Uref)
end

In [ ]:
function cost(ztraj)
    #Add Xref and Uref
    z = reshape(ztraj,Nx+Nu,Nt)
    J = 0.0
    for k = 1:Nt
        J += stage_cost(z[1:Nx,k],z[(Nx+1):(Nx+Nu),k],Xref_[k],Uref[k])
    end
    return J
end

In [ ]:
function dynamics_constraint!(c,ztraj)
    d = reshape(c,Nx,Nt-1)
    z = reshape(ztraj,Nx+Nu,Nt)
    for k = 1:(Nt-1)
        x1 = z[1:Nx,k]
        u1 = z[(Nx+1):(Nx+Nu),k]
        x2 = z[1:Nx,k+1]
        d[:,k] = dircol_dynamics(x1,x2,u1)
    end
    return nothing
end

In [ ]:
function con!(c,ztraj)
    z = reshape(ztraj,Nx+Nu,Nt)#7 x 101
    #Periodicity Constraint 
    #Xf - X0 = 0 
    c[1:Nx] .= z[1:Nx,end] - z[1:Nx,1]
    @views dynamics_constraint!(c[(Nx+1):(end-Nx)],ztraj)
end

In [ ]:
struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

#Our code 
function primal_bounds(n)
    x_l = -Inf*ones(n)
    x_u = Inf*ones(n) #+_ 20% dt_ref
    for k = 1:Nt-1
        x_l[u_idx[k]] .= 0 #Something reasonable based on dt_ref 0.8*dt
    end 

    return x_l, x_u
end

#Our code 
function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)
    c_u = zeros(m)
    return c_l, c_u
end
#Our code 

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    cost(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,cost,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    con!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), con!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-6,c_tol=1.0e-6,max_iter=10000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

In [ ]:
#Initial and goal states
x0 = Xref_[1]
xgoal = Xref_[end]

In [ ]:
#Guess
z0 = zeros((Nx+Nu)*Nt)
for k=1:Nt
    z0[x_idx[k]] = Xref_[k]
    z0[u_idx[k]] = Uref[k]
end
@show size(z0)

In [ ]:
prob = ProblemMOI(n_nlp,m_nlp)
z_sol = solve(z0,prob) # solve
# ztraj = reshape(z_sol,Nx+Nu,Nt)
# xtraj = ztraj[1:Nx,:]
# utraj = ztraj[(Nx+1):(Nx+Nu),:]

In [ ]:
plot(thist,xtraj[1,:])
plot(thist,xtraj[2,:])
plot(thist,xguess[1,:])
PyPlot.show()

In [ ]:
plot(thist,xtraj[3,:])
plot(thist,xtraj[4,:])
PyPlot.show()

In [ ]:
plot(thist,utraj[1,:])
PyPlot.show()

In [ ]:
using TrajOptPlots
using MeshCat
using StaticArrays

vis = Visualizer()
render(vis)

In [ ]:
TrajOptPlots.set_mesh!(vis, a)

In [ ]:
X1 = [SVector{4}(x) for x in eachcol(xtraj)];
visualize!(vis, a, thist[end], X1)